# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
# Importando as bibliotecas que irei usar.

import pandas as pd
import numpy as np
from seaborn import load_dataset
import seaborn as sns
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [5]:
#Vamos realizar antes de tudo a limpeza do banco de dados.
#Devido a grande extensao da base, irei remover as linhas com dados faltantes.

df = df.dropna()

#Removendo linhas duplicadas.

df = df.drop_duplicates()

#Resetando o index.

df = df.reset_index()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12427 entries, 0 to 12426
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  12427 non-null  int64  
 1   Unnamed: 0             12427 non-null  int64  
 2   data_ref               12427 non-null  object 
 3   id_cliente             12427 non-null  int64  
 4   sexo                   12427 non-null  object 
 5   posse_de_veiculo       12427 non-null  bool   
 6   posse_de_imovel        12427 non-null  bool   
 7   qtd_filhos             12427 non-null  int64  
 8   tipo_renda             12427 non-null  object 
 9   educacao               12427 non-null  object 
 10  estado_civil           12427 non-null  object 
 11  tipo_residencia        12427 non-null  object 
 12  idade                  12427 non-null  int64  
 13  tempo_emprego          12427 non-null  float64
 14  qt_pessoas_residencia  12427 non-null  float64
 15  re

In [11]:
#Analisando a frequência das categorias em cada variável:

for coluna in df.columns[3:]:
    print(coluna, '\n', df[coluna].value_counts(), '\n')

id_cliente 
 id_cliente
8635     6
9948     6
9351     6
5573     6
6356     6
        ..
7969     1
6978     1
13650    1
9869     1
12939    1
Name: count, Length: 8161, dtype: int64 

sexo 
 sexo
F    7901
M    4526
Name: count, dtype: int64 

posse_de_veiculo 
 posse_de_veiculo
False    7160
True     5267
Name: count, dtype: int64 

posse_de_imovel 
 posse_de_imovel
True     8116
False    4311
Name: count, dtype: int64 

qtd_filhos 
 qtd_filhos
0     7887
1     2973
2     1358
3      183
4       17
7        5
14       2
5        2
Name: count, dtype: int64 

tipo_renda 
 tipo_renda
Assalariado         7633
Empresário          3508
Servidor público    1268
Bolsista               9
Pensionista            9
Name: count, dtype: int64 

educacao 
 educacao
Secundário             7045
Superior completo      4695
Superior incompleto     558
Primário                103
Pós graduação            26
Name: count, dtype: int64 

estado_civil 
 estado_civil
Casado      8897
Solteiro    1543
Uniã

In [7]:
#Com a extensao da base é muito grande, vou fazer a modelagem apenas com a segunda parte.
#Primeiro vamos fazer uma cópia da base de dados.

df_1 = df.copy()

#Depois, selecionando a segunda parte da base de dados.

df_1 = df_1.iloc[6213:,:]

In [8]:
#Base de dados pronta, inicio a modelagem o patsy.
#As variáveis sexo, tipo_renda, educacao, estado_civil e tipo_residencia são variáveis categóricas.
#A casela de referência é a categoria com maior frequencia.

y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + \
                       C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia, Treatment("Casa")) + idade + tempo_emprego + qt_pessoas_residencia + 0', data=df_1)

In [9]:
#Com o statsmodel vizualizamos como ficou a regressão.
#Podemos notar que as variáveis utilizadas para explicar o modelo de regressão fazem sentido.

modelo = sm.OLS(y, X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     143.1
Date:                Sat, 27 Jul 2024   Prob (F-statistic):               0.00
Time:                        08:25:41   Log-Likelihood:                -6841.3
No. Observations:                6214   AIC:                         1.373e+04
Df Residuals:                    6189   BIC:                         1.390e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
C(sexo, Treatment("F"))[F]                                      6.2230      0.291     21.361      0.000       5.652       6.794
C(sexo, Treatment("F"))[M]                                      7.0246      0.291     24.111      0.000       6.453       7.596
posse_de_veiculo[T.True]                                        0.0591      0.020      2.930      0.003       0.020       0.099
posse_de_imovel[T.True]                                         0.0796      0.020      3.966      0.000       0.040       0.119
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.3055      0.327      0.934      0.350      -0.335       0.946
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1564      0.021      7.350      0.000       0.115       0.198
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2950      0.422     -0.700      0.484      -1.122       0.532
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0630      0.032      1.995      0.046       0.001       0.125
C(educacao, Treatment("Secundário"))[T.Primário]               -0.1441      0.101     -1.422      0.155      -0.343       0.055
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.0857      0.189      0.453      0.651      -0.285       0.457
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.0748      0.019      3.856      0.000       0.037       0.113
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0379      0.045     -0.845      0.398      -0.126       0.050
C(estado_civil, Treatment("Casado"))[T.Separado]                0.4138      0.149      2.783      0.005       0.122       0.705
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.4069      0.145      2.800      0.005       0.122       0.692
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0326      0.035     -0.929      0.353      -0.101       0.036
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.5202      0.155      3.366      0.001       0.217       0.823
C(tipo_residencia, Treatment("Casa"))[T.Aluguel]               -0.0022      0.078     -0.028      0.978      -0.155       0.151
C(tipo_residencia, Treatment("Casa"))[T.Com os pais]            0.0157      0.042      0.377      0.706      -0.066       0.097
C(tipo_residencia, Treatment("Casa"))[T.Comunitário]           -0.0712      0.132     -0.539      0.590      -0.330       0.188
C(tipo_residencia, Treatment("Casa"))[T.Estúdio]                0.0901      0.118      0.766      0.444      -0.141       0.321
C(tipo_residencia, Treatment("Casa"))[T.Governamental]          0.0048      0.055      0.087      0.931      -0.103       0.113
qtd_filhos 

### Observação:

A variável tipo_residencia não apresentou significância estatística para o modelo, tendo em vista que o p-value é maior que 0.05 em todas as categorias.

Olhando a variável R² ajustada, apresentou uma melhora mínima enquanto o AIC diminuiu minimamente. Acredito que o modelo ficou melhor dado que foi eiminado uma variável que não apresenta tanto relevância para o modelo.

In [10]:
#Removendo tipo_residencia do modelo.

y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + \
                       C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + idade + tempo_emprego + qt_pessoas_residencia + 0', data=df_1)

modelo = sm.OLS(y, X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     180.8
Date:                Sat, 27 Jul 2024   Prob (F-statistic):               0.00
Time:                        08:28:20   Log-Likelihood:                -6841.8
No. Observations:                6214   AIC:                         1.372e+04
Df Residuals:                    6194   BIC:                         1.386e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
C(sexo, Treatment("F"))[F]                                      6.2244      0.291     21.384      0.000       5.654       6.795
C(sexo, Treatment("F"))[M]                                      7.0270      0.291     24.143      0.000       6.456       7.598
posse_de_veiculo[T.True]                                        0.0592      0.020      2.943      0.003       0.020       0.099
posse_de_imovel[T.True]                                         0.0783      0.020      3.977      0.000       0.040       0.117
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.3062      0.327      0.937      0.349      -0.334       0.947
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1568      0.021      7.387      0.000       0.115       0.198
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2963      0.422     -0.703      0.482      -1.123       0.530
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0640      0.032      2.028      0.043       0.002       0.126
C(educacao, Treatment("Secundário"))[T.Primário]               -0.1447      0.101     -1.429      0.153      -0.343       0.054
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.0848      0.189      0.448      0.654      -0.286       0.456
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.0742      0.019      3.833      0.000       0.036       0.112
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0377      0.045     -0.842      0.400      -0.125       0.050
C(estado_civil, Treatment("Casado"))[T.Separado]                0.4151      0.149      2.793      0.005       0.124       0.706
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.4075      0.145      2.805      0.005       0.123       0.692
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0321      0.035     -0.915      0.360      -0.101       0.037
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.5194      0.154      3.363      0.001       0.217       0.822
qtd_filhos                                                     -0.3878      0.143     -2.704      0.007      -0.669      -0.107
idade                                                           0.0067      0.001      5.830      0.000       0.004       0.009
tempo_emprego                                                   0.0599      0.001     40.522      0.000       0.057       0.063
qt_pessoas_residencia                                           0.4204      0.143      2.936      0.003       0.140       0.701
==============================================================================
Omnibus:                        0.580   Durbin-Watson:      

### Observação

A variável educação nos apresenta quatro categorias e apenas uma delas é significante estatísticamente para o modelo. Dessa forma ela também será removida do modelo.

In [12]:
# Removendo educacao do modelo

y, X = patsy.dmatrices('np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment("Assalariado")) + \
                       C(estado_civil, Treatment("Casado")) + idade + tempo_emprego + qt_pessoas_residencia + 0', data=df_1)

modelo = sm.OLS(y, X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     227.0
Date:                Sat, 27 Jul 2024   Prob (F-statistic):               0.00
Time:                        08:35:44   Log-Likelihood:                -6852.2
No. Observations:                6214   AIC:                         1.374e+04
Df Residuals:                    6198   BIC:                         1.384e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
C(sexo, Treatment("F"))[F]                                      6.2757      0.291     21.583      0.000       5.706       6.846
C(sexo, Treatment("F"))[M]                                      7.0726      0.291     24.321      0.000       6.503       7.643
posse_de_veiculo[T.True]                                        0.0646      0.020      3.212      0.001       0.025       0.104
posse_de_imovel[T.True]                                         0.0810      0.020      4.117      0.000       0.042       0.120
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.3535      0.327      1.081      0.280      -0.288       0.995
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1632      0.021      7.707      0.000       0.122       0.205
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.2518      0.422     -0.597      0.551      -1.079       0.575
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0749      0.031      2.380      0.017       0.013       0.137
C(estado_civil, Treatment("Casado"))[T.Separado]                0.4066      0.149      2.733      0.006       0.115       0.698
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.3964      0.145      2.725      0.006       0.111       0.681
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0352      0.035     -1.003      0.316      -0.104       0.034
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.5009      0.155      3.240      0.001       0.198       0.804
qtd_filhos                                                     -0.3733      0.144     -2.600      0.009      -0.655      -0.092
idade                                                           0.0066      0.001      5.839      0.000       0.004       0.009
tempo_emprego                                                   0.0600      0.001     40.557      0.000       0.057       0.063
qt_pessoas_residencia                                           0.4080      0.143      2.846      0.004       0.127       0.689
==============================================================================
Omnibus:                        0.868   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.648   Jarque-Bera (JB):                0.821
Skew:                           0.013   Prob(JB):                        0.663
Kurtosis:                       3.050   Cond. No.                     2.38e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+03. This might indicate that there a

### Observação

Remover a variavel educacao fez com que o R²  tivessem valores menores, mas o AIC maior.

As variáveis restantes apresentam mais categorias com significância estatística.

In [13]:
#Removendo as variáveis que continuam apresentam p-value maior que 5%.

smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel
                                + tipo_renda == 'Empresário'
                                + tipo_renda == 'Servidor público'
                                + educacao == 'Superior completo'
                                + qt_pessoas_residencia == 3
                                + qt_pessoas_residencia == 4
                                + idade 
                                + tempo_emprego
                ''', 
        data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     688.2
Date:                Sat, 27 Jul 2024   Prob (F-statistic):               0.00
Time:                        08:39:53   Log-Likelihood:                -13576.
No. Observations:               12427   AIC:                         2.717e+04
Df Residuals:                   12416   BIC:                         2.726e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    7.0815      0.035    201.800      0.000       7.013       7.150
sexo[T.M]                                    0.7844      0.015     53.961      0.000       0.756       0.813
posse_de_veiculo[T.True]                     0.0437      0.014      3.117      0.002       0.016       0.071
posse_de_imovel[T.True]                      0.0824      0.014      6.011      0.000       0.055       0.109
tipo_renda == 'Empresário'[T.True]           0.1534      0.015     10.318      0.000       0.124       0.182
tipo_renda == 'Servidor público'[T.True]     0.0572      0.022      2.578      0.010       0.014       0.101
educacao == 'Superior completo'[T.True]      0.1083      0.014      7.979      0.000       0.082       0.135
qt_pessoas_residencia == 3[T.True]           0.0443      0.017      2.596      0.009       0.011       0.078
qt_pessoas_residencia == 4[T.True]           0.0843      0.022      3.806      0.000       0.041       0.128
idade                                        0.0060      0.001      7.783      0.000       0.004       0.008
tempo_emprego                                0.0615      0.001     59.297      0.000       0.059       0.064
==============================================================================
Omnibus:                        0.888   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.641   Jarque-Bera (JB):                0.862
Skew:                           0.018   Prob(JB):                        0.650
Kurtosis:                       3.018   Cond. No.                         241.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Resposta:

Ao comparar o modelo final com o modelo inicial, os indicadores não apresentaram nenhuma melhora ou piora significativa. O coeficiente de determinação diminuiu de 0.358 para 0.357 e o AIC de 27190 para 27170.
